# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import gspread

## Data read-in

In [3]:
df = pd.read_csv('Austin luxury map - 12_16.csv')

In [4]:
df

,Address,Price,Price psf
0,"2104 Island Wood Rd Austin, Texas 78733",$4.95 million,"$1,006"
1,"4901 Peralta Ln Austin, Texas 78735",$4.25 million,$894
2,"5814 Trailridge Dr, Austin, Texas 78731",$3.6 million,$709
3,"6108 Mountainclimb Dr, Austin, Texas 78731",$2.85 million,$744
4,"7913 Brightman Ln, Austin, Texas 78733",$2.5 million,$528
5,"8204 Hickory Creek Dr, Austin, Texas 78735",$2.35 million,$648
6,"1710 Palma Plz, Austin, Texas 78703",$2.2 million,$852


In [5]:
# df['Description'].iloc[0]

## Geocode

In [5]:
df.columns

Index(['Address', 'Price', 'Price psf'], dtype='object')

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [7]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Address'].apply(geocode)

In [8]:
# Convert 'geocoded' column into Shapely Point objects
df['geometry'] = df['geocoded'].apply(lambda coords: Point(coords[1], coords[0]))

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

,Address,Price,Price psf,geocoded,geometry
0,"2104 Island Wood Rd Austin, Texas 78733",$4.95 million,"$1,006","(30.337473, -97.8476592)",POINT (-97.84766 30.33747)
1,"4901 Peralta Ln Austin, Texas 78735",$4.25 million,$894,"(30.2622931, -97.87434300000001)",POINT (-97.87434 30.26229)
2,"5814 Trailridge Dr, Austin, Texas 78731",$3.6 million,$709,"(30.3418244, -97.7584292)",POINT (-97.75843 30.34182)
3,"6108 Mountainclimb Dr, Austin, Texas 78731",$2.85 million,$744,"(30.3476547, -97.7642297)",POINT (-97.76423 30.34765)
4,"7913 Brightman Ln, Austin, Texas 78733",$2.5 million,$528,"(30.3192302, -97.84546069999999)",POINT (-97.84546 30.31923)
5,"8204 Hickory Creek Dr, Austin, Texas 78735",$2.35 million,$648,"(30.2809165, -97.86600290000001)",POINT (-97.86600 30.28092)
6,"1710 Palma Plz, Austin, Texas 78703",$2.2 million,$852,"(30.2833637, -97.76166309999999)",POINT (-97.76166 30.28336)


In [9]:
gdf.columns

Index(['Address', 'Price', 'Price psf', 'geocoded', 'geometry'], dtype='object')

In [10]:
gdf = gdf.drop(columns={'geocoded':'Details'})

In [11]:
# Strip whitespace from all string columns
gdf.columns = gdf.columns.str.strip()  # Trim whitespace from column names
gdf = gdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Trim whitespace from string values

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_67422/2323350875.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gdf = gdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Trim whitespace from string values


In [12]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [13]:
gdf.to_file('data.geojson', driver='GeoJSON')

In [14]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Austin_Luxury_Map_12_16_24


In [15]:
gdf.columns

Index(['Address', 'Price', 'Price psf', 'geometry'], dtype='object')

<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>